<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/intermediate-results/notebooks/4.0b-mb-gbm_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost==1.1
!pip install gcsfs==2022.10.0
# !pip install ipywidgets==8.0.2
!pip install numpy==1.23.4
!pip install pandas==1.5.1
!pip install scikit-learn==1.1.3
!pip install fastparquet
!pip install -U ipywidgets==7.7.1
!jupyter nbextension enable --py widgetsnbextension

Requirement already up-to-date: ipywidgets==7.7.1 in /usr/local/lib/python3.8/dist-packages (7.7.1)
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


Run `pip install .` first to install all dependencies.

In [2]:
from catboost import CatBoostClassifier, Pool, metrics

import gcsfs
# import google.auth
# from google.colab import auth, output

import numpy as np
import pandas as pd

import os
from pathlib import Path

from sklearn.metrics import accuracy_score

#from tqdm.notebook import tqdm
# import wandb

from typing import List, Optional

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
# connect to google cloud storage
# auth.authenticate_user()
# credentials, _ = google.auth.default()
# fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)
gcsfs.GCSFileSystem(project="thesis")
fs_prefix = "gs://"

/usr/lib/python3/dist-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
features_classical_size = [
    'TRADE_PRICE', 'bid_ask_size_ratio_ex', 'rel_bid_size_ex',
       'rel_ask_size_ex', 'TRADE_SIZE', 'bid_size_ex', 'ask_size_ex',
       'rel_ask_ex', 'rel_bid_ex', 'BEST_rel_bid', 'BEST_rel_ask',
       'bid_ask_ratio_ex', 'chg_ex_lead', 'chg_ex_lag', 'chg_all_lead',
       'chg_all_lag', 'ask_ex', 'bid_ex', 'BEST_ASK', 'BEST_BID',
       'price_all_lag', 'price_all_lead', 'price_ex_lag', 'price_ex_lead'
]

In [5]:
# output_path = (
#     f"gs://thesis-bucket-option-trade-classification/data/classical_size_features_log_normalized/train_set_60.parquet"
# )

# output_path = (
#     f"gs://thesis-bucket-option-trade-classification/data/classical_size_features_log_normalized/val_set_20.parquet"
# )

X_train = pd.read_parquet("gs://thesis-bucket-option-trade-classification/data/classical_size_features_log_normalized/train_set_60.parquet", engine="fastparquet")
y_train = X_train["buy_sell"]
X_train = X_train[features_classical_size]

X_val = pd.read_parquet("gs://thesis-bucket-option-trade-classification/data/classical_size_features_log_normalized/val_set_20.parquet", engine="fastparquet")
y_val = X_val["buy_sell"]
X_val = X_val[features_classical_size]

/usr/lib/python3/dist-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## CatBoost Baseline 🐈‍⬛

In [6]:
model = CatBoostClassifier(task_type="GPU", iterations=10, custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent')
model.fit(X_train, y_train, eval_set=(X_val, y_val), plot=True);

/usr/local/lib/python3.8/dist-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [27]:
acc = model.score(X_val, y_val)

print(acc)

0.7170233763713466
